In [ ]:
import pickle
import numpy as np
import pandas as pd
from project_functions.sample_feed_v1_multi import SampleFeed
from project_functions import baseline
import matplotlib.pyplot as plt
from importlib import reload

from keras.models import load_model
from keras import losses
from keras import optimizers
from keras import metrics

from modeling_v1_run import TRAINING_WINDOW_SIZE, PREDICTED_WINDOW_SIZE, N_SAMPLES

## Final model selection

In [ ]:
# Examine model history

with open("models/history/comb_history0101.pk", "rb") as file:
    model_history = pickle.load(file)

plt.plot(model_history.history['loss'], color='black')
plt.plot(model_history.history['val_loss'], color='blue')

In [ ]:
df_history = pd.DataFrame(model_history.history)
df_history['epoch'] = pd.to_numeric(df_history.index) + 1

In [ ]:
# Select the best performing model

df_history.sort_values('val_root_mean_squared_error')[0:10]

In [ ]:
# Load the chosen model

model = load_model("models/checkpoints/comb_0101_45-0.080647.keras")

# Final model evaluation

In [ ]:
# Build the test dataset (not used for model validation)

features_test = dict(np.load("data/features_test.npz", allow_pickle=True))

with open("data/visits_scaler_test.pk", "rb") as file:
    visits_scaler_test = pickle.load(file)

sample_feed = SampleFeed(
    training_window_size = TRAINING_WINDOW_SIZE,
    predicted_window_size = PREDICTED_WINDOW_SIZE,
    samples_per_epoch = N_SAMPLES
    )

Xy_test = sample_feed.random_sample_array(features_test, samples_per_page=1, shuffle=False, seed=0)

In [ ]:
# Evaluate model on the test dataset

model.evaluate(*Xy_test)

In [ ]:
# Make predictions and inverse scaling

y_pred = model.predict(Xy_test[0])

X_test_rescaled = visits_scaler_test. \
    inverse_transform(Xy_test[0][0][:,:,0])

y_test_rescaled = visits_scaler_test. \
    inverse_transform(Xy_test[1])

y_pred_rescaled = visits_scaler_test. \
    inverse_transform(y_pred)

In [ ]:
# Utils for plotting

def one_page_results(no):

    X_emp_i = X_test_rescaled[no,:]
    y_emp_i = y_test_rescaled[no,:]
    y_pred_i = y_pred_rescaled[no,:]

    xy_emp = np.concatenate([X_emp_i, y_emp_i.reshape(PREDICTED_WINDOW_SIZE)])
    xy_pred = np.concatenate([np.full_like(X_emp_i[:-1], np.nan), X_emp_i[-1:], y_pred_i.reshape(PREDICTED_WINDOW_SIZE)])

    return xy_emp, xy_pred

def results_gen():

    ips_arr = np.array(range(Xy_test[0][0].shape[0]))

    np.random.seed(0)
    np.random.shuffle(ips_arr)

    for ip in ips_arr:
        yield ip, one_page_results(ip)

def plot_results(xy_scaled_emp, xy_scaled_pred):

    plt.figure(figsize=(20, 8))
    plt.plot(xy_scaled_emp, color="black")
    plt.plot(xy_scaled_pred, color="blue")
    plt.show()

results_show_iter = results_gen()

In [ ]:
# Browsing results for different Wiki pages

ip, (xy_scaled_emp, xy_scaled_pred) = next(results_show_iter)

print(ip)
plot_results(xy_scaled_emp, xy_scaled_pred)

In [ ]:
# Select representative examples for results description

representative_examples = [
    44148, 61555, 49345, 42420, 5726,
    9709, 66087, 17384, 2534, 17719,
    45459, 18972, 6287, 23952, 33636, 
    36134, 36972, 56528, 9675, 18508, 
    14268, 25413, 34399, 51223, 3066, 
    26959, 26382, 44801, 64919, 18078
    ]

In [ ]:
# Plot with some examples for results description

x_subplots = 6
y_subplots = 5

fig, axs = plt.subplots(x_subplots, y_subplots, sharex=True, sharey=False, figsize=(4*x_subplots,4*y_subplots))

ips = iter(representative_examples)

for ix in range(x_subplots):
    for iy in range(y_subplots):
        
        ip = next(ips)
        
        X_emp_i = X_test_rescaled[ip,:]
        y_emp_i = y_test_rescaled[ip,:]
        y_pred_i = y_pred_rescaled[ip,:]

        xy_emp = np.concatenate([X_emp_i, y_emp_i.reshape(PREDICTED_WINDOW_SIZE)])
        xy_pred = np.concatenate([np.full_like(X_emp_i[:-1], np.nan), X_emp_i[-1:], y_pred_i.reshape(PREDICTED_WINDOW_SIZE)])

        axs[ix,iy].plot(xy_emp, color="black")
        axs[ix,iy].plot(xy_pred, color="blue")

## Baseline models

In [ ]:
# Build and evaluate the Naive Model

naive_model = baseline.NaiveModel(PREDICTED_WINDOW_SIZE)

naive_model.compile(
    loss=losses.Huber(0.25), 
    optimizer=optimizers.Adam(learning_rate=1e-3), 
    metrics=metrics.RootMeanSquaredError()
    )

y_pred_naive = naive_model.predict(Xy_test[0][0])

X_test_rescaled = visits_scaler_test. \
    inverse_transform(Xy_test[0][0][:,:,0])

y_test_rescaled = visits_scaler_test. \
    inverse_transform(Xy_test[1])

y_pred_naive_rescaled = visits_scaler_test. \
    inverse_transform(y_pred_naive)

In [ ]:
naive_model.evaluate(Xy_test[0][0], Xy_test[1])

In [ ]:
# Build and evaluate the Random Model

random_model = baseline.RandomModel(PREDICTED_WINDOW_SIZE)

random_model.compile(
    loss=losses.Huber(0.25), 
    optimizer=optimizers.Adam(learning_rate=1e-3), 
    metrics=metrics.RootMeanSquaredError()
    )

y_pred_random = random_model.predict(Xy_test[0][0])

X_test_rescaled = visits_scaler_test. \
    inverse_transform(Xy_test[0][0][:,:,0])

y_test_rescaled = visits_scaler_test. \
    inverse_transform(Xy_test[1])

y_pred_random_rescaled = visits_scaler_test. \
    inverse_transform(y_pred_random)

In [ ]:
random_model.evaluate(Xy_test[0][0], Xy_test[1])

In [ ]:
# Plot baseline models

x_subplots = 5
y_subplots = 5

fig, axs = plt.subplots(x_subplots, y_subplots, sharex=True, sharey=False, figsize=(4*x_subplots,4*y_subplots))

ips = iter(representative_examples)

y_pred_random_rescaled

for ix in range(x_subplots):
    for iy in range(y_subplots):
        
        ip = next(ips)
        
        X_emp_i = X_test_rescaled[ip,:]
        y_emp_i = y_test_rescaled[ip,:]
        y_pred_naive_i = y_pred_naive_rescaled[ip,:]
        y_pred_random_i = y_pred_random_rescaled[ip,:]

        xy_emp = np.concatenate([X_emp_i, y_emp_i.reshape(PREDICTED_WINDOW_SIZE)])
        xy_pred_naive = np.concatenate([np.full_like(X_emp_i[:-1], np.nan), X_emp_i[-1:], y_pred_naive_i.reshape(PREDICTED_WINDOW_SIZE)])
        xy_pred_random = np.concatenate([np.full_like(X_emp_i[:-1], np.nan), X_emp_i[-1:], y_pred_random_i.reshape(PREDICTED_WINDOW_SIZE)])

        axs[ix,iy].plot(xy_emp, color="black")
        axs[ix,iy].plot(xy_pred_random, color=(0.9, 0.6, 0.1))
        axs[ix,iy].plot(xy_pred_naive, color=(0.9, 0.1, 0.0))